In [480]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep  5 09:46:49 2018
@author: Lucas de Moura Rodrigues
@ra: 14.00556-5
"""

'\nCreated on Wed Sep  5 09:46:49 2018\n@author: Lucas de Moura Rodrigues\n@ra: 14.00556-5\n'

In [481]:
import time
import numpy as np
import pandas as pd
from sklearn import linear_model,metrics,svm
import sklearn
import matplotlib.pyplot as plt

In [482]:
tempo_inicial = time.time()

In [483]:
# le o csv
print('Le e trata os dados: ', round(time.time() - tempo_inicial, 3))
precos_casa = pd.read_csv("precos_casa_california.csv")

Le e trata os dados:  0.014


In [484]:
# ve as informacoes do Dataframe
precos_casa.describe()

,latitude,median_income
count,20445.000000,20444.000000
mean,35.643596,3.876869
std,2.129236,1.904054
min,32.540000,0.499900
25%,33.940000,2.565750
50%,34.260000,3.540300
75%,37.720000,4.756600
max,41.950000,15.000100


In [485]:
# trata o dataframe, tirando outliers
precos_casa = precos_casa.replace(' ', np.nan)
precos_casa = precos_casa.dropna()
colunas = ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
for i in colunas:
    precos_casa[i] = pd.to_numeric(precos_casa[i])
    precos_casa = precos_casa.loc[precos_casa[i] < precos_casa[i].quantile(0.95)]
precos_casa = precos_casa.reset_index(drop=True)
precos_casa.describe()

,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,13930.000000,13930.000000,13930.000000,13930.000000,13930.000000,13930.000000,13930.000000,13930.000000
mean,35.694599,28.550323,1955.799282,400.786935,1097.638335,374.421608,3.515953,173585.886360
std,2.181956,11.149611,918.063132,174.487808,492.718730,160.755241,1.377513,80856.189371
min,32.540000,1.000000,2.000000,2.000000,3.000000,2.000000,0.499900,0.000000
25%,33.930000,19.000000,1307.250000,275.000000,744.000000,261.000000,2.453700,108000.000000
50%,34.300000,30.000000,1851.000000,387.000000,1057.000000,364.000000,3.360700,162500.000000
75%,37.720000,36.000000,2529.000000,520.000000,1431.000000,488.000000,4.440525,226500.000000
max,41.950000,51.000000,5989.000000,994.000000,2400.000000,735.000000,7.358800,387500.000000


In [486]:
# separa o dataframe
dados = precos_casa.loc[:, precos_casa.columns != 'median_house_value']
target = precos_casa.loc[:, precos_casa.columns == 'median_house_value']

In [487]:
dados.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,NEAR BAY
1,-122.26,37.85,50.0,1120.0,283.0,697.0,264.0,2.1250,NEAR BAY
2,-122.26,37.84,50.0,2239.0,455.0,990.0,419.0,1.9911,NEAR BAY


In [488]:
target.head(3)

,median_house_value
0,226700.0
1,140000.0
2,158700.0


In [489]:
# separa o dataframe: dados_treino = 0 - 60%  dados_validacao = 60 - 100%
tamanho_precos_casa = len(precos_casa)
index_60 = int(tamanho_precos_casa * 0.6)  # primeiros 60% do dataframe
dados_treino = dados[:index_60]
dados_validacao = dados[index_60:]
target_treino = target[:index_60]
target_validacao = target[index_60:]
dados_treino = dados_treino.sort_values(by=['median_income','households']).reset_index(drop=True)
dados_validacao = dados_validacao.sort_values(by=['median_income','households']).reset_index(drop=True)
target_validacao = target_validacao.sort_values(by='median_house_value').reset_index(drop=True)
target_treino = target_treino.sort_values(by='median_house_value').reset_index(drop=True)

In [490]:
dados_treino.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-118.04,33.90,36.0,15.0,5.0,15.0,6.0,0.4999,<1H OCEAN
1,-122.29,37.81,46.0,12.0,4.0,18.0,7.0,0.4999,NEAR BAY
2,-122.89,39.42,16.0,411.0,114.0,26.0,19.0,0.4999,INLAND
3,-119.99,36.09,23.0,333.0,92.0,198.0,55.0,0.4999,INLAND
4,-118.44,34.04,16.0,18.0,6.0,3.0,4.0,0.5360,<1H OCEAN


In [491]:
dados_validacao.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-120.19,38.07,43.0,102.0,19.0,44.0,13.0,0.4999,INLAND
1,-116.9,34.69,10.0,337.0,102.0,108.0,50.0,0.4999,INLAND
2,-123.32,40.43,15.0,661.0,146.0,131.0,57.0,0.4999,INLAND
3,-114.62,33.62,26.0,18.0,3.0,5.0,3.0,0.5360,INLAND
4,-116.76,34.14,4.0,42.0,10.0,9.0,3.0,0.5360,INLAND


In [492]:
target_treino.head()

,median_house_value
0,0.0
1,0.0
2,14999.0
3,14999.0
4,17500.0


In [493]:
target_validacao.head()

,median_house_value
0,14999.0
1,27500.0
2,32500.0
3,32500.0
4,34200.0


In [494]:
# numeriza o ocean
dic_ocean = {0: 'NEAR BAY', 1: '<1H OCEAN', 2: 'INLAND', 3: 'NEAR OCEAN', 4: 'ISLAND'}
for chave, valor in dic_ocean.items():
    dados_treino = dados_treino.replace(valor, chave)
    dados_validacao = dados_validacao.replace(valor, chave)

In [495]:
# machine learning
print('Predicao dos Dados: ', round(time.time() - tempo_inicial, 3))
#clf = linear_model.LassoLars(alpha=.1)  # 25480756266.08841
#clf = linear_model.Ridge (alpha = .5) # 25490381018.173256
clf = svm.SVR() # 6922054605.659882
#clf = linear_model.SGDRegressor() # infinito
clf.fit(dados_treino.values, target_treino.values.ravel())
target_predicao = clf.predict(dados_validacao)

Predicao dos Dados:  0.724


In [496]:
target_validacao = target_validacao.sort_values(by=['median_house_value']).reset_index(drop=True)
target_validacao.head()

,median_house_value
0,14999.0
1,27500.0
2,32500.0
3,32500.0
4,34200.0


In [497]:
target_predicao = [float(i) for i in target_predicao]
target_predicao = pd.DataFrame(data=target_predicao,columns=['median_house_value']).sort_values(by=['median_house_value'],ascending=False).reset_index(drop=True)
target_predicao.head()

,median_house_value
0,166099.478307
1,166099.472393
2,166099.472292
3,166099.472063
4,166099.472060


In [498]:
#sklearn.metrics.mean_squared_error(target_validacao, target_predicao)

In [499]:
# compara os dados
print('Comparando os Dados: ', round(time.time() - tempo_inicial, 3))
resultado = target_validacao.reset_index(drop=True).copy()
resultado['predicao'] = target_predicao['median_house_value']
resultado = resultado.rename(index=str, columns={'median_house_value': 'Valor Correto', 'predicao': 'Valor_Calculado'})
resultado['Diferenca_Absoluta'] = resultado['Valor Correto'] - resultado['Valor_Calculado']
resultado['Diferenca_Porcentagem'] = (resultado['Valor Correto'] / resultado['Valor_Calculado'] - 1) * 100
resultado.head()

Comparando os Dados:  8.486


,Valor Correto,Valor_Calculado,Diferenca_Absoluta,Diferenca_Porcentagem
0,14999.0,166099.478307,-151100.478307,-90.969869
1,27500.0,166099.472393,-138599.472393,-83.443656
2,32500.0,166099.472292,-133599.472292,-80.433412
3,32500.0,166099.472063,-133599.472063,-80.433412
4,34200.0,166099.472060,-131899.472060,-79.409929


In [500]:
# salva o resultado em um excel
excel = pd.ExcelWriter(F"Resultado.xlsx")
precos_casa.to_excel(excel, sheet_name='Precos_Casa_Tratado')
resultado.to_excel(excel, sheet_name='Comparacao')
excel.save()

In [501]:
print('Tempo do Programa: ', round(time.time() - tempo_inicial, 3))

Tempo do Programa:  12.029
